# Software Development und Python Basics

In diesem Notebook sammle ich ein paar Basics zum Entwickeln eines Projektes in Python.
Da ihr schon Kurse mit Python hattet, sollte das meiste nicht neu sein, eher sollen wichtige Parts wiederholt werden.
Wenn für euch noch wichtige Themen fehlen, kann ich diese hier noch hinzufügen. 

In [1]:
import datetime
import random
from pathlib import Path

import pandas as pd

## Organisierung eines Python Projekts

Eine kurze Übersicht, wie man seinen Code organisieren kann, von ersten Schritten und einfachen Programmen zu Komponenten die gut wiederverwendet werden können.

### Notebook

Als erste Methode, um schnell Sachen auszuprobieren und Ergebnisse sofort anzuzeigen, eignen sich Notebooks sehr gut.
Wenn immer wieder aufgeräumt wird und auch die anderen Features wie Markdown Überschriften und Text verwendet werden,
eignen sich diese auch sehr gut zum Anschaulich machen der einzelnen Schritte und um neues zu testen.
Da man aber oft an unterschiedlichen Stellen weiterarbeitet oder etwas neues probiert, können diese auch recht
schnell sehr unübersichtlich werden.

In [2]:
# Tipp um eigene Funktionen automatisch neu zu laden
%load_ext autoreload
%autoreload 2

### Skript

Wenn der Code, wie im Notebook definiert, in ein Python File kopiert wird, so dass dieser von oben nach unten
ausgeführt wird, hat man ein Skript. In den meisten Fällen werdet ihr die Hauptschritte eures Projektes
in einem Skript ausführen. Idealerweise werden die wichtigen Komponenten hier aber nur aufgerufen, 
und befinden sich schön gegliedert in anderen Files.

### Funktionen

Sobald die gewünschte Herangehensweise im Notebook ausgetestet wurde, macht es Sinn diese in 
einzelne Funktionen aufzuteilen. Diese haben jeweils eine klar beschrieben Aufgabe, definierte 
Input und Output Parameter und können somit auch wiederverwendet werden. 

Meine Empfehlung ist es, sich bevor man den Code in die Funktion gibt, zu überlegen welche Ein- und
Ausgangsparameter man erwartet. Am besten geht das, in dem man zuerst die Signatur der Funktion
inklusive Docstring schreibt, und dann die Funktionalität aus dem Notebook überträgt, oft mit
einigen Verallgemeinerungen.

In [3]:
# Example of a Function stub, to define functionality, in and output parameters before the implementation
def load_profile_for_day(  # Descriptive Name
    day: datetime.date,
    profile_df: pd.DataFrame,
    dynamize: bool = False,  # Parameters, with type hints to make explicit what we expect
) -> pd.Series:  # Return type
    # Docstring with one title line, optionally some more description, and a description of Arguments and what is returned
    """Get the 15 minute load profile for a single day.

    Args:
        day (datetime.date): Date for which to get the load profile.
        profile_df (pd.DataFrame): In the format as given by get_profile_from_file.
        dynamize (bool): Should dynamization by the 4th-grade polynomial dynamization
            function for day of year be applied. This should be only done for H0. Defaults to False.

    Returns:
        pd.Series: Series with Watt values every 15 minutes, and a pd.DatetimeIndex.
    """
    pass  # Implementation follows in second step.

Diese Herangehensweise habe ich für die Standardlastprofile gewählt. Im Notebook *11_load_profiles.ipynb*
habe ich die einzelnen Schritte ausprobiert, dann habe ich eine Funktion erstellt und diese 
schlussendlich in ein eigenes File *src/load_profiles.py* kopiert. Von dort können sie nun importiert werden.

In [4]:
from src.load_profiles import get_load_profile

In [5]:
lastprofile_file = Path(".") / "data" / "representative_profiles_vdew.xls"
assert (
    lastprofile_file.is_file()
), f"Did not find file with representative load profiles at {lastprofile_file}"

In [6]:
h0profile = get_load_profile(
    lastprofile_file,
    from_=datetime.date(2015, 1, 1),
    to=datetime.date(2015, 12, 31),
    type="H0",
)

In [ ]:
h0profile.head()

### Classes

Objekt-orientiertes Programmieren baut sehr stark auf Klassen auf. Vor allem nützlich wenn Objekte
erstellt werden sollen, die auch ihren eigenen Zustand speichern sollen.
Klasse am besten in einem eigenen File erstellen, hier ein kurzes Beispiel als Wiederholung.

In [8]:
# Class Names by convention in CamelCase
# Inheriting from default class object
class Wuerfel(object):
    # properties can be defined statically here
    seiten = [1, 2, 3, 4, 5, 6]

    # __init__ method is called when the class is instantiated
    # Usually used to define properties dynamically
    # Every method has self as it's first argument, this is used to access all class properties and methods
    def __init__(self, init_seite_oben: int = None):
        if init_seite_oben is None:
            self.seite_oben = random.choice(self.seiten)
        else:
            self.seite_oben = init_seite_oben

    # methods, self and possibly more arguments, optional return
    def wurf(self):
        self.seite_oben = random.choice(self.seiten)

    def setzen(self, seite_oben: int):
        self.seite_oben = seite_oben

    def check_seite_oben(self) -> bool:
        if self.seite_oben in self.seiten:
            return True
        else:
            return False

In [ ]:
# Instantiate 1 die
d1 = Wuerfel(1)
d1.seite_oben

In [ ]:
d1.wurf()
d1.seite_oben

In [ ]:
# Can create multiple instances of dice
d2 = Wuerfel()
d2.seite_oben

In [ ]:
# d1 is independent
d1.seite_oben

In [ ]:
print(d2.check_seite_oben())
d2.setzen(7)
print(d2.check_seite_oben())

In [ ]:
# Inspect class instance
dir(d2)
# __name__ methods are thunder methods that are defined by object. They are used for various operations
# Sometimes you want to redefine them, so you get special behaviour.
# _name methods and properties are usually private, so not intended to be used
# all our defined properties and methods are in the end

In [ ]:
# Overwrite the str method, that defines what happens when the class is printed / converted to string
print(d2)

In [16]:
def wuerfel_str(self):
    return str(self.seite_oben)


# Adding methods outside of the class definition is not good practice
Wuerfel.__str__ = wuerfel_str

In [ ]:
print(d2)

## Timeseries

Wir arbeiten mit Zeitserien, Diskretisierung in Intervalle, zB 15 Minuten Werte.

Python Tool: pandas DataFrame (oder Series) mit DateTimeIndex. 

### Daten importieren

Ich importiere hier die Daten die ich in Notebooks 10 und 11 erzeugt habe.
Wenn dieser Code nicht funktioniert, erst diese beiden Notebooks ausführen.

In [ ]:
pv = pd.read_csv("outputs/pv_example.csv")
pv.head()

In [ ]:
# May also directly read in index
g0 = pd.read_csv("outputs/g0_2015.csv", index_col=0)
g0.head()

### DatetimeIndex und Zeitzonen

Ein großer Vorteil von Pandas ist, das komplexere Objekte den Index bilden können, so zum Beispiel Datetimes.
Diese können verwendet werden um relativ einfach unterschiedliche Zeiträume auszuwählen, Plots haben per Default 
eine Zeitachse, und auch der Wechsel zwischen unterschiedlichen Intervallen oder Zusammenfassungen sind relativ simpel möglich.
Dazu zuerst ein paar Beispiele mit den Lastdaten von g0.

Vor allem wenn Daten aus verschiedenen Quellen verwendet werden, ist es wichtig das die Zeiten zueinander stimmen.
Durch Zeitzonen, Sommerzeit und unterschiedliche Datumsformate können da aber recht leicht Fehler auftreten.
Auch dazu will ich ein paar Code Beispiele geben.

In [ ]:
# Check if index is already DatetimeIndex
type(g0.index)

In [ ]:
# This does not work without a datetimeindex
g0.index.month
g0.loc["2015-05-01"]

In [ ]:
# This works in many cases, for non-standard formats see further below
g0.index = pd.to_datetime(g0.index)
type(g0.index)

In [ ]:
# Now this works
g0.index.month.unique()

In [ ]:
# And this
g0.loc["2015-01-01 07:00":"2015-01-01 10:00"]

In [ ]:
# And plotting with date axis
g0.loc["2015-04-01"].plot()
# TODO: Probably still an error in the data here, only one day with 'weekend' profile...

In [ ]:
# As the pv data is hourly, let's resample our demand data to hours too
g0.sum() / 4.0  # This should give 1000 kWh -> g0 is in W

In [29]:
# We resample by the mean as we have power units (would sum for energy)
g0_hourly = g0.resample("h").mean()

In [ ]:
# And now calculate Energy consumption by month
# Check https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects for the offset strings
...

In [ ]:
# Check different datetime strings
print(pv.dtypes)
# This works again, and also is in UTC (nice!)
pd.to_datetime(pv["time(UTC)"])

In [ ]:
# But let's suppose a different format
pv["time2"] = pd.to_datetime(pv["time(UTC)"]).dt.strftime("%d.%m.%y %H:%M")
pv["time2"].head()

In [ ]:
# As the warning states, this may not always work. In this case it mixed up month and day.
pd.to_datetime(pv["time2"][23:26])

In [ ]:
# So better to explicitely specify the format
# Format according to the C standard formats: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
pd.to_datetime(pv["time2"][23:26], format="%d.%m.%y %H:%M")

In [ ]:
# Even better to always use ISO 8601 format with time zone information
pv.index = pd.to_datetime(pv["time(UTC)"])
pv = pv.tz_localize("UTC")
pv.head()

In [ ]:
# We also have to use load profiles with timezone now! Should be changed in functionality, as this is in local time
g0.tz_localize("Europe/Berlin")

In [ ]:
g0 = g0.tz_localize("Europe/Berlin", ambiguous="NaT", nonexistent="shift_forward")
g0.head()

In [ ]:
# Handling missing values. After this suboptimal transformation, we have missing values now
print(g0.index.isna().sum())
g0h = g0.resample("H").mean()
g0h.isna().sum()

In [ ]:
g0h = g0h.ffill()  # Here we use Forward fill, may use bfill or value
# g0h = g0h.fillna(0)
g0h.isna().sum()

In [ ]:
# Make one DF with pv and g0 data
df = pd.DataFrame({"PV": pv["0"], "G0": g0h.squeeze()})
df.head()

### Grafische Libraries zum Arbeiten mit Pandas

* mito / mitosheet
* bamboolib
* pandasgui

Ich habe mitosheet getestet und glaube dass man damit ganz gut seinen Datenworkflow starten kann.
Eine Sache die nicht so gut unterstützt wird sind Indizes.

## Visualisierung

Möglichkeiten:

* matplotlib
* seaborn
* plotly

## Daten

* [APCS](https://www.apcs.at/de/clearing/technisches-clearing/lastprofile)
* [OPSD](https://open-power-system-data.org/)
* https://solar.htw-berlin.de/elektrische-lastprofile-fuer-wohngebaeude/
* https://www.renewables.ninja/
* https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis_en
* https://simbench.de/de/datensaetze/

## Libraries

### Anbindung Wiener Netze Smart Meter Daten

Wiener Netze Smart Meter Portal https://smartmeter-web.wienernetze.at/#/welcome.

Git Repository fuer Abruf ueber API: https://github.com/platysma/vienna-smartmeter

## Best Practices

* Docstrings
* Type Hints
* Start with Notebook, move important pieces to .py file
* black autoformatter
* Exception Handling
* autoreload